## Часть 1
Напишите скрипт, который скачивает все данные с выборов губернатора СПб 2019г для всех избирательных участков. 

Входная точка [по ссылке](http://notelections.online/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222). Затем нужно перейти на сайты территориальных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название ТИК (территориальная избирательная комиссия, некоторое объединение избирательных участков на определенной территории)
- номер УИК (избирательный участок)
- 14 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

In [8]:
import bs4
import pandas as pd
import requests

import numpy as np
%config InlineBackend.figure_format = 'svg'

In [9]:
def data_t():
    url_ind = 'http://notelections.online/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222'
    response = requests.get(url=url_ind)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all("table")
    table_rows = table[7].find_all('tr')
    
    name_lines = ['Номер ТИК', 'Номер УИК', ]
    for i in table_rows:
        if i.find_all('td')[1].contents :
            if i.find_all('td')[1].find('b') == None :
                name_lines += i.find_all('td')[1].contents[0]
    table_rows = table[8].find('tr').find_all('a')

    flag = 0
    
    for i in table_rows:
        
        if i.contents ==['Цифровые избирательные участки']:
            break
            
        response_i = requests.get(url= 'http://notelections.online' + i.get('href'))
        soup_i = bs4.BeautifulSoup(response_i.text, 'html.parser')
        table_i = soup_i.find_all("table")
        table_rows_i = table_i[6].find('tr').find_all('a')
        
        N_ps = []
        for j in table_rows_i:
            N_ps.append(j.contents[0])
        
        table_rows_i = table_i[8].find_all('b')
        line = []
        for j in table_rows_i:
            line.append(int(j.contents[0]))
        
        line =  [i.contents[0] for j in range(len(N_ps))] + N_ps + line  
        
        line=np.array(line)
        line = np.reshape(line, ( len(name_lines),len(N_ps)))
        
        if flag == 0:
            df_all = pd.DataFrame(line, index= name_lines)
            flag = 1
        else:
            df_all=pd.merge(left=df_all,right=pd.DataFrame(line, index= name_lines), left_index=True, right_index=True)
            
    return df_all


In [10]:
df_p = data_t()

In [11]:
df_p.to_csv("../data/task5_data.csv")
df_p

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,40_y,41_y,42_y,43_y,44_y,45,46_y,47,48_y,49_y
Номер ТИК,Территориальная избирательная комиссия №1,Территориальная избирательная комиссия №1,Территориальная избирательная комиссия №1,Территориальная избирательная комиссия №1,Территориальная избирательная комиссия №1,Территориальная избирательная комиссия №1,Территориальная избирательная комиссия №1,Территориальная избирательная комиссия №1,Территориальная избирательная комиссия №1,Территориальная избирательная комиссия №1,...,Территориальная избирательная комиссия №30,Территориальная избирательная комиссия №30,Территориальная избирательная комиссия №30,Территориальная избирательная комиссия №30,Территориальная избирательная комиссия №30,Территориальная избирательная комиссия №30,Территориальная избирательная комиссия №30,Территориальная избирательная комиссия №30,Территориальная избирательная комиссия №30,Территориальная избирательная комиссия №30
Номер УИК,УИК №1,УИК №2,УИК №3,УИК №4,УИК №5,УИК №6,УИК №7,УИК №8,УИК №9,УИК №10,...,УИК №2322,УИК №2323,УИК №2347,УИК №2348,УИК №2349,УИК №2350,УИК №2351,УИК №2352,УИК №2355,УИК №2356
"Число избирателей, внесенных в список избирателей на момент окончания голосования",1803,1466,2092,1056,1827,1557,2095,1523,1463,1629,...,192,196,247,230,380,284,132,320,485,620
"Число избирательных бюллетеней, полученных участковой избирательной комиссией",1200,1100,1600,1000,1400,1200,1600,1100,1300,1200,...,250,250,300,250,400,300,150,350,500,650
"Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования",587,433,576,318,495,453,532,590,473,486,...,56,127,158,137,285,166,86,217,335,210
"Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования",11,14,22,4,8,13,17,13,26,7,...,107,16,17,6,21,53,27,23,12,215
Число погашенных избирательных бюллетеней,602,653,1002,678,897,734,1051,497,801,707,...,87,107,125,107,94,81,37,110,153,225
"Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования",11,14,22,4,8,13,17,13,26,7,...,107,16,17,6,21,53,27,23,12,215
"Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования",587,430,576,318,493,453,532,590,473,486,...,56,127,158,137,285,166,86,217,335,210
Число недействительных избирательных бюллетеней,23,19,15,13,16,12,13,19,17,14,...,2,4,2,3,5,0,2,4,2,3
